In [1]:
import numpy as np
import cv2
import os
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
#Load MobileNetv2 Model
baseModel = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(128, 128, 3)
)

In [3]:
#build custom head
x = baseModel.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation="softmax")(x)
model = Model(inputs=baseModel.input, outputs=predictions)

In [4]:
#Freeze Base Layers

for layer in baseModel.layers:
    layer.trainable = False

In [5]:
#compile Model
INIT_LR = 1e-4
EPOCHS = 20
BS = 32
DIRECTORY = r"E:\AI_Course\Deep Learning\Face-Mask-Detection-master\dataset"
CATEGORIES = ["with_mask", "without_mask"]
INIT_LR = 1e-4
opt =Adam(learning_rate=INIT_LR)
model.compile(
    loss="binary_crossentropy",
    optimizer=opt,
    metrics=["accuracy"]
)

In [6]:
#Data Generator
trainAug = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

trainGen = trainAug.flow_from_directory("dataset",
                                        target_size=(224, 224),
                                        batch_size=BS,
                                        class_mode="categorical"
                                       )

Found 3846 images belonging to 2 classes.


In [7]:
#Train the Model
history = model.fit(
    trainGen,
    steps_per_epoch=trainGen.samples //
    BS,
    epochs=EPOCHS
)

Epoch 1/20
 59/120 [=============>................] - ETA: 52s - loss: 0.5755 - accuracy: 0.7140

C:\Users\Admin\anaconda3\envs\aidl\lib\site-packages\PIL\Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


120/120 [==============================] - 107s 870ms/step - loss: 0.4605 - accuracy: 0.8002
Epoch 2/20
120/120 [==============================] - 885s 7s/step - loss: 0.2157 - accuracy: 0.9434
Epoch 3/20
120/120 [==============================] - 109s 908ms/step - loss: 0.1476 - accuracy: 0.9615
Epoch 4/20
120/120 [==============================] - 1724s 14s/step - loss: 0.1136 - accuracy: 0.9685
Epoch 5/20
120/120 [==============================] - 106s 883ms/step - loss: 0.0948 - accuracy: 0.9743
Epoch 6/20
120/120 [==============================] - 2862s 24s/step - loss: 0.0836 - accuracy: 0.9743
Epoch 7/20
120/120 [==============================] - 108s 900ms/step - loss: 0.0719 - accuracy: 0.9809
Epoch 8/20
120/120 [==============================] - 154s 1s/step - loss: 0.0627 - accuracy: 0.9837
Epoch 9/20
120/120 [==============================] - 155s 1s/step - loss: 0.0575 - accuracy: 0.9830
Epoch 10/20
120/120 [==============================] - 175s 1s/step - loss: 0.0534 - a

In [9]:
#FaceMaskDetection(openCV)
#!pip install opencv-python matplotlib
faceNet = cv2.dnn.readNet(
    "deploy.prototxt",
    "res10_300x300_ssd_iter_140000.caffemodel"
)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    blob = cv2.dnn.blobFromImage(
        frame, 1.0, (300, 300),
        (104.0, 177.0, 123.0)

    )

    faceNet.setInput(blob)
    detections = faceNet.forward()

    for i in range(0,directions.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array(
                [frame.shape[1], 
frame.shape[0],
                 frame.shape[1],
frame.shape[0]]
            )
            (starX, starY, endX, endY) = box.astype("int")
            face = frame[starY:endY, starX:endX]
            face = cv2.resize(face,(224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            face = np.expand_dims(face, axis=0)
            (mask, noMask) = model.predict(face)[0]
            label = "Mask" if mask > noMask else "No Mask"
            
            color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
            cv2.putText(frame, label, (startX, startY - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.45, color,2)
            cv2.rectangle(frame, (startX, startY),
                          (endX, endY), color, 2)
            cv2.imshow("Mask Detector", frame)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break


cap.release()
cv2.destroyAllWindows()
            
                   
                   
    

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\caffe\caffe_io.cpp:1126: error: (-2:Unspecified error) FAILED: fs.is_open(). Can't open "deploy.prototxt" in function 'cv::dnn::ReadProtoFromTextFile'
